In [ ]:
#pip install nltk

In [ ]:
#pip install spacy

In [16]:
#import libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import sklearn


[nltk_data] Downloading package punkt to /Users/henry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/henry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
data = pd.DataFrame(pd.read_csv('Twitter Sentiment/cleaned_text.csv'))

def helper(string):
    temp = []
    for word in string[1:-1].split(','):
        word = word.strip()
        word = word[1:-1]
        if word:
          temp.append(word)  
    return temp
data.filtered_tokens = data.filtered_tokens.apply(helper)

In [29]:
#data = data.iloc[25000:75000]
#data.shape

(50000, 8)

In [34]:
# One hot encoding
categories = list(set([item for sublist in data.filtered_tokens for item in sublist]))
#print(categories)

vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in data['filtered_tokens']]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)
#print(categories)


       clocked  clauses  cara  jinx  languishing  movie  knackerrred  ads  \
0            0        0     0     0            0      0            0    0   
1            0        0     0     0            0      0            0    0   
2            0        0     0     0            0      0            0    0   
3            0        0     0     0            0      0            0    0   
4            0        0     0     0            0      0            0    0   
...        ...      ...   ...   ...          ...    ...          ...  ...   
99995        0        0     0     0            0      0            0    0   
99996        0        0     0     0            0      0            0    0   
99997        0        0     0     0            0      0            0    0   
99998        0        0     0     0            0      0            0    0   
99999        0        0     0     0            0      0            0    0   

       rig  envelopes  ...  unzip  moved  someonee  folks  soda  tiiiired  

In [31]:
count = (data['labels'] == 0).sum()
print(count)

25000


In [35]:
data = data.iloc[25000:75000]
encoded_df = encoded_df.iloc[25000:75000]

print(data.shape)
print(encoded_df.shape)

(50000, 8)
(50000, 50661)


In [36]:
# Split the dataset

X_train, X_test, y_train, y_test=train_test_split(encoded_df, data['labels'], stratify=data['labels'], test_size=0.2, random_state=42)

In [37]:
print(y_train)
len(y_train)

73903    1
35174    0
46519    0
34815    0
31895    0
        ..
65542    1
36522    0
48743    0
42799    0
38724    0
Name: labels, Length: 40000, dtype: int64


40000

In [38]:
#Logistic Regression 

def sigmoid(x):
    return 1/(1+np.exp(-x))

class LogisticRegression():

    def __init__(self, LR = 0.01, n_iterations = 1000):
        self.LR = LR
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape 
        self.weights = np.zeros(n_features)
        self.bias = 0 

        for i in range(self.n_iterations):
            linear_prediction = np.dot(X, self.weights) + self.bias
            predictions = sigmoid(linear_prediction)

            dw = (1/n_samples) * np.dot(X.T, (predictions - y))
            db = (1/n_samples) * np.sum(predictions - y)

            self.weights = self.weights - self.LR * dw
            self.bias = self.bias - self.LR * db 

    def predict(self, X):
        linear_prediction = np.dot(X, self.weights) + self.bias 
        y_pred = sigmoid(linear_prediction)
        class_pred = [0 if y <= 0.5 else 1 for y in y_pred]

        return class_pred

In [39]:
clf = sklearn.linear_model.LogisticRegression(random_state = 42).fit(X_train, y_train)


/Users/henry/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
y_pred = clf.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.729
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.69      0.72      1000
           1       0.71      0.77      0.74      1000

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

